In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import numpy as np
import cv2

In [6]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=config)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

/media/giorgos/external/UBUNTU/anaconda3/envs/deep_learning/lib/python3.8/site-packages/tensorflow/python/client/session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [7]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
data,labels = np.load("data/data.npy"),np.load("data/labels.npy")

In [9]:
xtrain,xtest,ytrain,ytest = train_test_split(data,labels,test_size = 0.2,shuffle=True,random_state=2)
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((1100, 128, 128, 1), (276, 128, 128, 1), (1100, 2), (276, 2))

In [10]:
class_indices = {0:"Mask",1:"No Mask"}

In [ ]:
image_dimensions = 128
batch_size = 8

nn = keras.models.Sequential()

nn.add(layers.Conv2D(filters=200,
                     kernel_size=(3,3),
                     strides=(1,1),
                     activation = 'relu',
                     input_shape=data.shape[1:]))

nn.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

nn.add(layers.Conv2D(filters=100,
                     kernel_size=(3,3),
                     activation = 'relu',
                     strides=(1,1)))
       
nn.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

       
nn.add(layers.Flatten())
       
nn.add(layers.Dense(100,activation = 'relu'))
nn.add(layers.Dense(2,activation = 'softmax'))
       
       
nn.compile(loss='categorical_crossentropy',optimizer = keras.optimizers.SGD(lr=0.001,momentum=0.8),metrics=['accuracy'])

In [ ]:
nn.fit(xtrain,
       ytrain,
       steps_per_epoch=len(xtrain) // batch_size,
       batch_size = batch_size,
       epochs = 15,
       validation_split=0.1)

In [ ]:
pred = nn.predict(xtest.reshape(len(xtest),128,128,1))
ytest = ytest.argmax(axis=1)
pred = pred.argmax(axis=1)
accuracy_score(ytest,pred)

In [ ]:
nn.save("model/nn.h5")